# Dose Response Study

Simulating dose response experiments.

We begin by importing required packages.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.optimize as opt
import statistics
from scipy import stats
import os
import re
import ptitprince as pt #rain plots
from datetime import date

import libraries.disturbances as dt
import libraries.normalization as nrm
import libraries.dose_response as dr
import libraries.utilities as util

In [ ]:
#absolute_ic50_data-36-8-dil8-1-0.4-20220413-right-half-neg-control-log-new-reg

## Data Generation

Here you can specify the particulars of the experiment.

In [2]:
## Simulation parameters
## Ensure the appropriate PLAID layouts exist under "layouts/compounds_PLAID_layouts"

concentrations = 8 #Possible to use 4, 6, 8
replicates = 1

# Total number of compounds that fit in a 384 well-plate after leaving the outer wells empty
# 
compounds = (14*22-20)//(concentrations*replicates)

if concentrations == 4:
    dilution = 15
elif concentrations == 6:
    dilution = 18
elif concentrations == 8:
    dilution = 8
elif concentrations == 12:
    dilution = 4
else:
    print("Please specify the dilution factor")

id_text = "delete"

#expected_noise = 0.01

error_nl = 0.4 

#my_min_dist = 0

In [3]:


#error_types = [{'type':"right-exp", 'error_function':dt.add_exponential_errors_to_right_columns,'error_correction':nrm.normalize_plate_column_effect,'error':error_nl}]
error_types = [{'type':"right-half", 'error_function':dt.add_errors_to_right_columns_half,'error_correction':nrm.normalize_plate_nearest_control,'error':error_nl}]
#error_types = [{'type':"diagonal", 'error_function':dt.add_diagonal_errors,'error_correction':nrm.normalize_plate_nearest_control,'error':error_nl}]
#error_types = [{'type':"bowl-nl", 'error_function':dt.add_bowlshaped_errors_nl,'error_correction':nrm.normalize_plate_nearest_control,'error':error_nl}]
               #{'type':"bowl", 'error_function':dt.add_bowlshaped_errors,'error_correction':nrm.normalize_plate_nearest_control,'error':error_l}]
               #{'type':"left-nl", 'error_function':dt.add_errors_to_left_columns,'error_correction':nrm.normalize_plate_column_effect,'error':error_nl},
               #{'type':"left", 'error_function':dt.add_linear_errors_to_left_columns,'error_correction':nrm.normalize_plate_column_effect,'error':error},
               #{'type':"right", 'error_function':dt.add_linear_errors_to_right_columns,'error_correction':nrm.normalize_plate_column_effect,'error':error},
               #{'type':"top-nl", 'error_function':dt.add_errors_to_upper_rows,'error_correction':nrm.normalize_plate_row_effect,'error':error_nl},
               #{'type':"top", 'error_function':dt.add_linear_errors_to_upper_rows,'error_correction':nrm.normalize_plate_row_effect,'error':error},
               #{'type':"bottom", 'error_function':dt.add_linear_errors_to_lower_rows,'error_correction':nrm.normalize_plate_row_effect,'error':error},
               #{'type':"left-exp", 'error_function':dt.add_exponential_errors_to_left_columns,'error_correction':nrm.normalize_plate_column_effect,'error':error_nl},
               #{'type':"top-exp", 'error_function':dt.add_exponential_errors_to_upper_rows,'error_correction':nrm.normalize_plate_row_effect,'error':error_nl}]
#{'type':,error_function':,'error_correction':}


plate_types_location = [{'type':'PLAID','dir':'layouts/compounds_PLAID_layouts/','regex':'plate_layout_(.*)'+str(compounds)+'-'+str(concentrations)+'-'+str(replicates)+'_(0*)(.+?).npy','error_correction':nrm.normalize_plate_lowess_2d},
                        {'type':'RANDOM','dir':'layouts/compounds_manual_layouts/','regex':'plate_layout_rand_(.+?).npy','error_correction':nrm.normalize_plate_lowess_2d},
                        {'type':'BORDER','dir':'layouts/compounds_manual_layouts/','regex':'plate_layout_border_(.+?).npy','error_correction':nrm.normalize_plate_linear}]





In [4]:
e_from=1
e_to=5##100
e_step=5


In [ ]:
#plate_types_location, error_types, e_from=1, e_to=100, e_step=5, lose_rows_from, lose_rows_to, 

In [ ]:
def full_dose_response_evaluation(plate_types_location, error_types, e_from=1, e_to=100, e_step=5, lose_rows_from=1, lose_rows_to=2):

today = (date.today()).strftime("%Y%m%d")+"-"+id_text

## Results
absolute_ic50_data_f=open('absolute_ic50_data-'+str(compounds)+'-'+str(concentrations)+'-dil'+str(dilution)+'-'+str(replicates)+'-'+str(error_nl)+'-'+today+'.csv','a')
relative_ic50_data_f=open('relative_ic50_data-'+str(compounds)+'-'+str(concentrations)+'-dil'+str(dilution)+'-'+str(replicates)+'-'+str(error_nl)+'-'+today+'.csv','a')
residuals_f=open('residuals-'+str(compounds)+'-'+str(concentrations)+'-dil'+str(dilution)+'-'+str(replicates)+'-'+str(error_nl)+'-'+today+'.csv','a')

    for current_e in range(e_from,e_to,e_step):
        print("\nTesting compounds with e in range ",current_e,"-",(current_e+e_step),":")

        # Create new curves/compounds

        params = dr.generate_compound_curves(compounds,concentrations,dilution,current_e)

        df_params = pd.DataFrame.from_dict(params)
        df_params.set_index('compound',inplace=True)
        df_params['abs IC50'] = dr.IC50(df_params['b'],df_params['c'],df_params['d'],df_params['e'])

        compounds_array = df_params.index.to_numpy()

        # The same compounds/concentrations will be used in every plate
        plate_content = dr.generate_plate_content(dose_response_params=params, replicates=replicates)

        for plate_type in plate_types_location:
            print("Using ",plate_type['type']," layouts...")
            layout_dir = plate_type['dir']
            layouts = os.listdir(layout_dir)

            for layout_file in layouts:
                match = re.search(plate_type['regex'],layout_file)

                # Skip this file if it doesn't match the regular expression in plate_type['regex']
                if match == None:
                    continue

                layout_file_array = np.full(len(compounds_array),layout_file)

                e_array = np.full(len(compounds_array),current_e)

                for et in error_types:
                    error_type_array = np.full(len(compounds_array),et['type'])
                    error_array = np.full(len(compounds_array),et['error'])    
                    for lost_rows in range(1,2): #max 8
                        limits = [#{'from':0, 'to':lost_rows},
                                  {'from':16-lost_rows,'to':16}]
                        for limit in limits:
                            function(compounds_array,lost_rows,layout_file,plate_content,et,plate_type,limit,absolute_ic50_data_f,relative_ic50_data_f,residuals_f)    


Now we place them on a plate, add a small random noise, apply plate effects error and correct them.

In [7]:
print("Results are being stored in files which include the name: ",str(compounds)+'-'+str(concentrations)+'-dil'+str(dilution)+'-'+str(replicates)+'-'+str(error_nl)+'-'+today)

full_dose_response_evaluation()

## Close all files
absolute_ic50_data_f.close()
relative_ic50_data_f.close()
residuals_f.close()

print("\nDone! :-)")       

Results are being stored in files which include the name:  36-8-dil8-1-0.4-20220426-delete

Testing compounds with e in range  1 - 6 :
Using  PLAID  layouts...


ValueError: I/O operation on closed file.

In [5]:
def function(compounds_array,lost_rows,layout_file,plate_content,et,plate_type,limit,absolute_ic50_data_f,relative_ic50_data_f,residuals_f,expected_noise=0.01):
    layout_dir = plate_type['dir']
    r_lost_array = np.full(len(compounds_array),lost_rows)

    fitTable_new = dr.plate_curves_after_error(layout_dir,layout_file,plate_content,expected_noise,et['error_function'],et['error'],plate_type['error_correction'],lose_from_row=limit['from'],lose_to_row=limit['to'])

    obtained_absolute_ic50 = dr.IC50(fitTable_new['b'],fitTable_new['c'],fitTable_new['d'],fitTable_new['e'])

    res_array = np.concatenate(fitTable_new['residuals'].to_numpy())
    true_res_array = np.concatenate(fitTable_new['true_residuals'].to_numpy())
    res_size = len(res_array)

    plate_residuals = np.vstack([np.full(res_size,layout_file), np.full(res_size,et['type']), np.full(res_size,et['error']), np.full(res_size,current_e), np.full(res_size,lost_rows), res_array, true_res_array])

    np.savetxt(residuals_f, plate_residuals.T, delimiter=",",fmt='%s')

    plate_rel_ic50 = np.absolute(np.subtract(np.log10(df_params['e']), np.log10(fitTable_new['e'])))
    plate_abs_ic50 = np.absolute(np.subtract(np.log10(df_params['abs IC50']), np.log10(obtained_absolute_ic50)))

    # Includes curve info
    rel_results = np.vstack([layout_file_array, compounds_array, plate_rel_ic50.to_numpy(), error_type_array, error_array, e_array, r_lost_array, fitTable_new['r2_score'],df_params['b'],df_params['c'],df_params['d'],df_params['e'],fitTable_new['b'],fitTable_new['c'],fitTable_new['d'],fitTable_new['e']])
    abs_results = np.vstack([layout_file_array, compounds_array, plate_abs_ic50.to_numpy(), error_type_array, error_array, e_array, r_lost_array, fitTable_new['r2_score'],df_params['b'],df_params['c'],df_params['d'],df_params['e'],fitTable_new['b'],fitTable_new['c'],fitTable_new['d'],fitTable_new['e']])

    # Save results
    np.savetxt(absolute_ic50_data_f, abs_results.T, delimiter=",",fmt='%s')
    np.savetxt(relative_ic50_data_f, rel_results.T, delimiter=",",fmt='%s')

In [ ]:
# don't go beyond here with Run All
assert False

## Evaluating the absolute IC50

We first look at the mean IC50 obtained for all the compounds using a PLAID layout versus a completly random layout. 

In [ ]:
## Loading data for a high error=0.125
# 1 replicate
#absolute_ic50_data = np.loadtxt('absolute_ic50_data-48-6-1-20210630.csv', delimiter=',', dtype='str')

# 2 replicate
#absolute_ic50_data = np.loadtxt('absolute_ic50_data-20210621.csv', delimiter=',', dtype='str')

# 3 replicates
#absolute_ic50_data = np.loadtxt('absolute_ic50_data-16-6-3complete-20210716.csv', delimiter=',', dtype='str')


## Loading data for a low error=0.025
# 1 replicate
#absolute_ic50_data = np.loadtxt('absolute_ic50_data-16-6-3-0.025-20210921.csv', delimiter=',', dtype='str')

## Loading data for a high error=0.125 and 4 doses
# 2 replicates
#absolute_ic50_data = np.loadtxt('absolute_ic50_data-36-4-2-8-20211212.csv', delimiter=',', dtype='str')

In [ ]:
results_df = pd.DataFrame(absolute_ic50_data, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])
results_df.MSE = pd.to_numeric(results_df.MSE, errors='coerce')
#results_df.MSE = pd.to_numeric(results_df.MSE, errors='coerce')

results_df = results_df.sort_values("MSE")

results_df = results_df[np.logical_not(np.isnan(results_df['MSE']))]

#results_df = results_df.loc[(results_df['error type'] != "bowl") & (results_df['error type'] != "left") & (results_df['error type'] != "top")]

print(results_df)

results_df.loc[(results_df['layout'] >= "plate_layout_rand"), 'layout'] = "RANDOM"
results_df.loc[(results_df['layout'] >= "plate_layout_border") & (results_df['layout'] != "RANDOM"), 'layout'] = "BORDER"
results_df.loc[(results_df['layout'] >= "plate_layout") & (results_df['layout'] != "RANDOM") & (results_df['layout'] != "BORDER"), 'layout'] = "TEST"


#print(results_df[results_df['layout']=='PLAID'].describe())

print(results_df[results_df['layout']=='RANDOM'].describe())

print(results_df[results_df['layout']=='BORDER'].describe())

fig, ax = plt.subplots(figsize=(4, 4))

ax.set(ylim=(0,1))


ax = sns.barplot(x="layout", y="MSE", data=results_df[results_df['MSE']!=np.inf], order=["TEST", "RANDOM", "BORDER"])

#ax = sns.barplot(x="layout", y="MSE", data=results_df[results_df['MSE']!=np.inf], order=["PLAID", "RANDOM"])

ax.set(xlabel='', ylabel='')

plt.show()


In [ ]:
fig.savefig("dose-response-absic50-48-4-2-sq-essense.png",bbox_inches='tight')

In [ ]:
## Plot Absolute IC50
results_df = pd.DataFrame(absolute_ic50_data, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])
results_df.MSE = pd.to_numeric(results_df.MSE, errors='coerce')

results_df["rows lost"] = pd.to_numeric(results_df["rows lost"], errors='coerce')
results_df["E"] = pd.to_numeric(results_df["E"], errors='coerce')

results_df = results_df.sort_values("MSE")

results_df = results_df[np.logical_not(np.isnan(results_df['MSE']))]
print(results_df)

results_df.loc[(results_df['layout'] >= "plate_layout_rand"), 'layout'] = "RANDOM"
results_df.loc[(results_df['layout'] >= "plate_layout_border") & (results_df['layout'] != "RANDOM"), 'layout'] = "BORDER"
results_df.loc[(results_df['layout'] >= "plate_layout") & (results_df['layout'] != "RANDOM") & (results_df['layout'] != "BORDER"), 'layout'] = "PLAID"

temp_none = results_df[(results_df['MSE']!=np.inf) & (results_df['rows lost']<2) ]

#print(temp_none[temp_none['layout']=='PLAID'].describe())

#print(temp_none[temp_none['layout']=='RANDOM'].describe())

#print(temp_none[temp_none['layout']=='BORDER'].describe())


#temp_half = results_df[(results_df['MSE']!=np.inf) & (results_df['rows lost']<=1) & (results_df['E']>70) & (results_df['E']<80) ]
temp_half = results_df[(results_df['rows lost']<=1) & (results_df['MSE']!=np.inf) ]
#ax = sns.barplot(x="layout", y="MSE", data=temp_half, order=["PLAID", "RANDOM"])
#plt.show()

print(temp_half[temp_half['layout']=='PLAID'].describe())

print(temp_half[temp_half['layout']=='RANDOM'].describe())

#print(temp_half[temp_half['layout']=='BORDER'].describe())


fig, ax = plt.subplots(figsize=(6, 15))

ax.set(ylim=(0,0.6))

ax = sns.barplot(x="layout", y="MSE", data=temp_half, order=["PLAID", "RANDOM","BORDER"])
#plt.show()

In [ ]:
fig.savefig("dose-response-absic50-16-6-3.png",bbox_inches='tight')

In [ ]:
#fig.savefig("dose-response-absic50-plaid-rand-36-4-2_0.125_0.25.png")

rand_results_array = results_df.MSE[results_df.layout=="RANDOM"]
plaid_results_array = results_df.MSE[results_df.layout=="PLAID"]
border_results_array = results_df.MSE[results_df.layout=="BORDER"]

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array))
print("Variance of BORDER layouts:", statistics.variance(border_results_array))

print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array))
print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array,equal_var = False))

print("RANDOM vs BORDER layouts:", stats.ttest_ind(border_results_array,rand_results_array,equal_var = False))

print("PLAID vs BORDER layouts:", stats.ttest_ind(plaid_results_array,border_results_array,equal_var = False))

In [ ]:
rand_results_array = temp_none.MSE[temp_none.layout=="RANDOM"]
plaid_results_array = temp_none.MSE[temp_none.layout=="PLAID"]
border_results_array = temp_none.MSE[temp_none.layout=="BORDER"]

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array))
print("Variance of BORDER layouts:", statistics.variance(border_results_array))

print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array))
print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array,equal_var = False))

print("RANDOM vs BORDER layouts:", stats.ttest_ind(border_results_array,rand_results_array,equal_var = False))

print("PLAID vs BORDER layouts:", stats.ttest_ind(plaid_results_array,border_results_array,equal_var = False))

In [ ]:
rand_results_array = temp_half.MSE[temp_half.layout=="RANDOM"]
plaid_results_array = temp_half.MSE[temp_half.layout=="PLAID"]
border_results_array = temp_half.MSE[temp_half.layout=="BORDER"]

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array))
print("Variance of BORDER layouts:", statistics.variance(border_results_array))

print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array))
print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array,equal_var = False))

print("RANDOM vs BORDER layouts:", stats.ttest_ind(border_results_array,rand_results_array,equal_var = False))

print("PLAID vs BORDER layouts:", stats.ttest_ind(plaid_results_array,border_results_array,equal_var = False))

### Plotting results for multiple replicates together

This is how we generate the plots for the paper.

In [ ]:
## Loading data for a high error=0.125
# 1 replicate
absolute_ic50_data_1rep = np.loadtxt('absolute_ic50_data-48-6-1-20210630.csv', delimiter=',', dtype='str')

# 2 replicate
absolute_ic50_data_2rep = np.loadtxt('absolute_ic50_data-20210621.csv', delimiter=',', dtype='str')

# 3 replicates
absolute_ic50_data_3rep = np.loadtxt('absolute_ic50_data-16-6-3complete-20210716.csv', delimiter=',', dtype='str')

In [ ]:
results_df = pd.DataFrame(absolute_ic50_data_1rep, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])

results_df_2rep = pd.DataFrame(absolute_ic50_data_2rep, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])

results_df_3rep = pd.DataFrame(absolute_ic50_data_3rep, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])

results_df.insert(0, 'replicates', 1)
results_df_2rep.insert(0, 'replicates', 2)
results_df_3rep.insert(0, 'replicates', 3)

results_df = results_df.append(results_df_2rep)
results_df = results_df.append(results_df_3rep)

results_df.MSE = pd.to_numeric(results_df.MSE, errors='coerce')

results_df = results_df[np.logical_not(np.isnan(results_df['MSE']))]

results_df.loc[(results_df['layout'] >= "plate_layout_rand"), 'layout'] = "RANDOM"
results_df.loc[(results_df['layout'] >= "plate_layout_border") & (results_df['layout'] != "RANDOM"), 'layout'] = "BORDER"
results_df.loc[(results_df['layout'] >= "plate_layout") & (results_df['layout'] != "RANDOM") & (results_df['layout'] != "BORDER"), 'layout'] = "PLAID"

fig, ax = plt.subplots(figsize=(6, 4))
ax.set(ylim=(0,0.6))
ax = sns.barplot(x='replicates', y="MSE", data=results_df[results_df['MSE']!=np.inf], hue="layout", palette='rocket_r')
plt.show()


In [ ]:
fig.savefig("dose-response-absic50-1-2-3.png",bbox_inches='tight')

## Evaluating the relative EC50/IC50

Now we evaluate the relative IC50's obtained. We plot the mean of the IC50 for all compounds obtained using either a PLAID layout or a random layout.

In [ ]:
## Data with lost rows
#relative_ic50_data = np.loadtxt('relative_ic50_data-20210624.csv', delimiter=',', dtype='str')

# 1 replicate
#relative_ic50_data = np.loadtxt('relative_ic50_data-48-6-1-20210630.csv', delimiter=',', dtype='str')

# 2 replicates
relative_ic50_data = np.loadtxt('relative_ic50_data-20210621.csv', delimiter=',', dtype='str')

# 3 replicates
#relative_ic50_data = np.loadtxt('relative_ic50_data-16-6-3complete-20210716.csv', delimiter=',', dtype='str')

In [ ]:
results_df = pd.DataFrame(relative_ic50_data, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])
results_df.MSE = pd.to_numeric(results_df.MSE, errors='coerce')
results_df = results_df.sort_values("MSE")

results_df["rows lost"] = pd.to_numeric(results_df["rows lost"], errors='coerce')
results_df["E"] = pd.to_numeric(results_df["E"], errors='coerce')

results_df = results_df.sort_values("MSE")

#results_df = results_df[(results_df['MSE']!=np.inf) & (results_df['rows lost']>1)]

## Rename the layouts so they can be grouped as PLAID, RANDOM or Border
results_df.loc[(results_df['layout'] >= "plate_layout_rand"), 'layout'] = "RANDOM"
results_df.loc[(results_df['layout'] >= "plate_layout_border") & (results_df['layout'] != "RANDOM"), 'layout'] = "BORDER"
results_df.loc[(results_df['layout'] >= "plate_layout") & (results_df['layout'] != "RANDOM"), 'layout'] = "PLAID"

results_df = results_df[(results_df['rows lost']<=1) ]

print(results_df[results_df['layout']=='PLAID'].describe())

print(results_df[results_df['layout']=='RANDOM'].describe())

print(results_df[results_df['layout']=='BORDER'].describe())

fig, ax = plt.subplots(figsize=(4, 4)) #6,15

ax.set(ylim=(0,0.42))

ax = sns.barplot(x="layout", y="MSE", data=results_df, order=["PLAID", "RANDOM", "BORDER"])

ax.set(xlabel='', ylabel='')
plt.show()

In [ ]:
fig.savefig("dose-response-relic50-24-6-2-sq.png",bbox_inches='tight')

In [ ]:

fig, ax = plt.subplots(figsize=(15, 8))

ax = sns.barplot(x="rows lost", y="MSE", hue="layout", data=results_df)

fig.savefig("dose-response-relic50-24-6-2_lost-rows.125_0.01.png")

In [ ]:
#fig.savefig("dose-response-relic50-plaid-rand-36-4-2_0.125_0.25.png")

rand_results_array = results_df.MSE[results_df.layout=="RANDOM"]
plaid_results_array = results_df.MSE[results_df.layout=="PLAID"]
border_results_array = results_df.MSE[results_df.layout=="BORDER"]

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array))
print("Variance of BORDER layouts:", statistics.variance(border_results_array),"\n")

print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array,equal_var = False))
print("RANDOM vs BORDER layouts:", stats.ttest_ind(border_results_array,rand_results_array,equal_var = False))
print("PLAID vs BORDER layouts:", stats.ttest_ind(plaid_results_array,border_results_array,equal_var = False))

In [ ]:
temp_half = results_df[(results_df['MSE']!=np.inf) & (results_df['rows lost']<=1) & (results_df['E']>20) & (results_df['E']<100) ]
#temp_half.loc[(temp_half['layout']=='PLAID') & (temp_half["MSE"]>0.688),"MSE"] = 0.1

fig, ax = plt.subplots(figsize=(15, 10))

#ax = sns.barplot(x="layout", y="MSE", data=temp_half, order=["PLAID", "RANDOM"])

#sns.barplot(x = "group", y = "score", data = df, capsize= .1)
pal = sns.color_palette(n_colors=1)

#ax=pt.half_violinplot( x = "MSE", y = "layout", data = temp_half, palette = pal, bw = .2, cut = 0.,
 #                     scale = "area", width = .6, inner = None, orient = ort, order=["PLAID", "RANDOM"])

dx = "MSE"; dy = "layout"; ort = "h"; pal = "Set2"; sigma = 0.01
#f, ax = plt.subplots(figsize=(7, 5))

pt.RainCloud(x = "layout", y = "MSE", data = temp_half, palette = pal, bw = sigma,
                 width_viol = 1.3, ax = ax, orient = ort, order=["PLAID", "RANDOM", "BORDER"], jitter = 0.15)

#ax = sns.violinplot(x="MSE", y="layout", data=temp_half, order=["PLAID", "RANDOM"])
#ax = sns.stripplot(x="MSE", y="layout", data=temp_half,morder=["PLAID", "RANDOM"], jitter=0.3)



In [ ]:
rand_results_array = temp_half.MSE[temp_half.layout=="RANDOM"]
plaid_results_array = temp_half.MSE[temp_half.layout=="PLAID"]
border_results_array = temp_half.MSE[temp_half.layout=="BORDER"]

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array))
print("Variance of BORDER layouts:", statistics.variance(border_results_array))

print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array))
print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array,equal_var = False))

print("RANDOM vs BORDER layouts:", stats.ttest_ind(border_results_array,rand_results_array,equal_var = False))

print("PLAID vs BORDER layouts:", stats.ttest_ind(plaid_results_array,border_results_array,equal_var = False))

### Plotting results for multiple replicates together

This is how we generate the plots for the paper.

In [ ]:
## Data with lost rows
#relative_ic50_data = np.loadtxt('relative_ic50_data-20210624.csv', delimiter=',', dtype='str')

# 1 replicate
relative_ic50_data_1rep = np.loadtxt('relative_ic50_data-48-6-1-20210630.csv', delimiter=',', dtype='str')

# 2 replicates
relative_ic50_data_2rep = np.loadtxt('relative_ic50_data-20210621.csv', delimiter=',', dtype='str')

# 3 replicates
relative_ic50_data_3rep = np.loadtxt('relative_ic50_data-16-6-3complete-20210716.csv', delimiter=',', dtype='str')

In [ ]:
results_df = pd.DataFrame(relative_ic50_data_1rep, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])
results_df_2rep = pd.DataFrame(relative_ic50_data_2rep, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])
results_df_3rep = pd.DataFrame(relative_ic50_data_3rep, columns=["layout", "method", "MSE", "error type", "Error", "E", "rows lost"])

results_df.insert(0, 'replicates', 1)
results_df_2rep.insert(0, 'replicates', 2)
results_df_3rep.insert(0, 'replicates', 3)

results_df = results_df.append(results_df_2rep)
results_df = results_df.append(results_df_3rep)

results_df.MSE = pd.to_numeric(results_df.MSE, errors='coerce')

results_df = results_df[np.logical_not(np.isnan(results_df['MSE']))]

results_df.loc[(results_df['layout'] >= "plate_layout_rand"), 'layout'] = "RANDOM"
results_df.loc[(results_df['layout'] >= "plate_layout_border") & (results_df['layout'] != "RANDOM"), 'layout'] = "BORDER"
results_df.loc[(results_df['layout'] >= "plate_layout") & (results_df['layout'] != "RANDOM") & (results_df['layout'] != "BORDER"), 'layout'] = "PLAID"

fig, ax = plt.subplots(figsize=(6, 4))
ax.set(ylim=(0,0.6))
ax = sns.barplot(x='replicates', y="MSE", data=results_df[results_df['MSE']!=np.inf], hue="layout", palette='rocket_r')
plt.show()


In [ ]:
fig.savefig("dose-response-relic50-1-2-3.png",bbox_inches='tight')

## Evaluating Residuals

We first look at the residuals against the fitted sigmoid

In [ ]:
# 1 replicate
residuals = np.loadtxt('residuals-48-6-1-20210630.csv', delimiter=',', dtype='str')

# 2 replicates
#residuals = np.loadtxt('residuals-20210621.csv', delimiter=',', dtype='str')

# 3 replicates
#residuals = np.loadtxt('residuals-16-6-3complete-20210716.csv', delimiter=',', dtype='str')


In [ ]:
residuals_df = pd.DataFrame(residuals, columns=["layout", "error_type", "Error", "E", "rows lost", "residuals", "true_residuals"])
residuals_df.residuals = pd.to_numeric(residuals_df.residuals, errors='coerce')
residuals_df.true_residuals = pd.to_numeric(residuals_df.true_residuals, errors='coerce')
residuals_df['rows lost'] = pd.to_numeric(residuals_df['rows lost'], errors='coerce')

residuals_df = residuals_df[(residuals_df['rows lost']<=1) ]

#residuals_df = residuals_df[np.logical_not(np.isnan(residuals_df["residuals"]))]

#residuals_df = residuals_df.sort_values("residuals")

## Rename the layouts so they can be grouped as PLAID, RANDOM or Border
residuals_df.loc[(residuals_df['layout'] >= "plate_layout_rand"), 'layout'] = "RANDOM"
residuals_df.loc[(residuals_df['layout'] >= "plate_layout_border") & (residuals_df['layout'] != "RANDOM"), 'layout'] = "BORDER"
residuals_df.loc[(residuals_df['layout'] >= "plate_layout") & (residuals_df['layout'] != "RANDOM"), 'layout'] = "PLAID"

print(residuals_df)

#res_data = np.empty((0,2))
#for layout in ['PLAID','RANDOM','BORDER']:
#    layout_res = np.concatenate(np.array(residuals_df[residuals_df['layout']==layout]['residuals']))
#    res = np.vstack([np.full(len(layout_res),layout),layout_res]).T
#    res_data = np.vstack([res_data,res])

#res_df = pd.DataFrame(res_data, columns=["layout", "SE"])
#res_df.SE = pd.to_numeric(res_df.SE, errors='coerce')
#res_df = res_df.sort_values("SE")

residuals_df.residuals = pd.to_numeric(residuals_df.residuals, errors='coerce')
residuals_df = residuals_df.sort_values("residuals")

print("PLAID:\n",residuals_df[residuals_df['layout']=='PLAID'].describe(),"\n")

print("RANDOM:\n",residuals_df[residuals_df['layout']=='RANDOM'].describe(),"\n")

print("BORDER:\n",residuals_df[residuals_df['layout']=='BORDER'].describe(),"\n")




In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

ax = sns.barplot(x="layout", y="true_residuals", data=residuals_df, order=["PLAID", "RANDOM", "BORDER"])

In [ ]:
ax = sns.barplot(x="layout", y="residuals", data=residuals_df, order=["PLAID", "RANDOM"])
plt.show()

ax = sns.barplot(x="layout", y="true_residuals", data=residuals_df, order=["PLAID", "RANDOM"])
plt.show()


In [ ]:
# Rain plot
temp_half = residuals_df#[(residuals_df['MSE']!=np.inf) & (residuals_df['rows lost']<=1) & (residuals_df['E']>20) & (residuals_df['E']<100) ]
#temp_half.loc[(temp_half['layout']=='PLAID') & (temp_half["MSE"]>0.688),"MSE"] = 0.1

fig, ax = plt.subplots(figsize=(15, 10))

#ax = sns.barplot(x="layout", y="MSE", data=temp_half, order=["PLAID", "RANDOM"])

#sns.barplot(x = "group", y = "score", data = df, capsize= .1)
pal = sns.color_palette(n_colors=1)

#ax=pt.half_violinplot( x = "MSE", y = "layout", data = temp_half, palette = pal, bw = .2, cut = 0.,
 #                     scale = "area", width = .6, inner = None, orient = ort, order=["PLAID", "RANDOM"])

dx = "MSE"; dy = "layout"; ort = "h"; pal = "Set2"; sigma = 0.01
#f, ax = plt.subplots(figsize=(7, 5))

ax = pt.RainCloud(x = "layout", y = "residuals", data = temp_half, palette = pal, bw = sigma,
                 width_viol = 1.2, ax = ax, orient = ort, order=["PLAID", "RANDOM", "BORDER"], jitter = 0.15)

#ax = sns.violinplot(x="MSE", y="layout", data=temp_half, order=["PLAID", "RANDOM"])
#ax = sns.stripplot(x="MSE", y="layout", data=temp_half,morder=["PLAID", "RANDOM"], jitter=0.3)

ax.set(xlabel='Residuals', ylabel='Layout type')

plt.show()

In [ ]:
fig.savefig("raincloud-dose-response-true-residuals-plaid-rand-border-20210621-36-4-2_0.125_0.25.png")

In [ ]:
# Rain plot

temp = residuals_df[["layout", "residuals", "true_residuals"]]#[(residuals_df['MSE']!=np.inf) & (residuals_df['rows lost']<=1) & (residuals_df['E']>20) & (residuals_df['E']<100) ]
#temp_half.loc[(temp_half['layout']=='PLAID') & (temp_half["MSE"]>0.688),"MSE"] = 0.1

temp_half = temp.melt('layout', var_name='res', value_name='vals')

#fig, ax = plt.subplots(figsize=(15, 10))

#ax = sns.barplot(x="layout", y="MSE", data=temp_half, order=["PLAID", "RANDOM"])

#sns.barplot(x = "group", y = "score", data = df, capsize= .1)
pal = sns.color_palette(n_colors=1)

#ax=pt.half_violinplot( x = "MSE", y = "layout", data = temp_half, palette = pal, bw = .2, cut = 0.,
 #                     scale = "area", width = .6, inner = None, orient = ort, order=["PLAID", "RANDOM"])

dx = "MSE"; dy = "layout"; ort = "h"; pal = "Set2"; sigma = 0.01
#f, ax = plt.subplots(figsize=(7, 5))

#temp_half_sample = temp_half.sample(n=300000)

# Rainclouds with FacetGrid
g = sns.FacetGrid(temp_half, col = "res", height = 5, aspect=1.4, margin_titles=False)
#g.set(ylim=(0, 55000))
g = g.map_dataframe(pt.RainCloud, x = "layout", y = "vals", data = temp_half, 
                    orient = "v", order=["BORDER", "RANDOM", "PLAID"], width_viol = 1.3)

axes = g.axes.flatten()
axes[0].set_title(" ")
axes[1].set_title(" ")

axes[0].set_ylabel(" ")
for ax in axes:
    ax.set_xlabel(" ")
    
#ax = pt.RainCloud(x = "layout", y = "vals", hue = 'res', data = temp_half, palette = pal, bw = sigma,
 #                width_viol = 1.2, ax = ax, orient = ort, order=["PLAID", "RANDOM", "BORDER"], jitter = 0.15)

#ax = sns.violinplot(x="res", y="layout", hue = , data=temp_half, order=["PLAID", "RANDOM"])
#ax = sns.stripplot(x="MSE", y="layout", data=temp_half,order=["PLAID", "RANDOM"], jitter=0.3)

#ax.set(xlabel='Residuals', ylabel='Layout type')

#plt.show()

In [ ]:
g.savefig("raincloud-dose-response-residuals-1-replicates-brp.png",bbox_inches='tight')

In [ ]:
rand_results_array = residuals_df.residuals[residuals_df.layout=="RANDOM"]
plaid_results_array = residuals_df.residuals[residuals_df.layout=="PLAID"]
border_results_array = residuals_df.residuals[residuals_df.layout=="BORDER"]

print("Mean of PLAID layouts:", statistics.mean(plaid_results_array))
print("Mean of RANDOM layouts:", statistics.mean(rand_results_array))
print("Mean of BORDER layouts:", statistics.mean(border_results_array),"\n")

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array))
print("Variance of BORDER layouts:", statistics.variance(border_results_array),"\n")

print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array,equal_var = False))
print("RANDOM vs BORDER layouts:", stats.ttest_ind(border_results_array,rand_results_array,equal_var = False))
print("PLAID vs BORDER layouts:", stats.ttest_ind(plaid_results_array,border_results_array,equal_var = False))

print("One way ANOVA:",stats.f_oneway(rand_results_array,plaid_results_array,border_results_array))

In [ ]:
#temp

rand_results_array = temp[temp.layout=="RANDOM"]
plaid_results_array = temp[temp.layout=="PLAID"]
border_results_array = temp[temp.layout=="BORDER"]

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array.residuals))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array.residuals))
print("Variance of BORDER layouts:", statistics.variance(border_results_array.residuals),"\n")

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array.true_residuals))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array.true_residuals))
print("Variance of BORDER layouts:", statistics.variance(border_results_array.true_residuals),"\n")

print("PLAID vs PLAID layouts:", stats.ttest_ind(plaid_results_array.residuals,plaid_results_array.true_residuals,equal_var = False))
print("RANDOM vs RANDOM layouts:", stats.ttest_ind(rand_results_array.residuals,rand_results_array.true_residuals,equal_var = False))
print("BORDER vs BORDER layouts:", stats.ttest_ind(border_results_array.residuals,border_results_array.true_residuals,equal_var = False))



We now look at the residuals compared to the sigmoid used to generated the data

In [ ]:
true_res_data = np.empty((0,2))
for layout in ['PLAID','RANDOM','BORDER']:
    layout_res = np.concatenate(np.array(residuals_df[residuals_df['layout']==layout]['true_residuals']))
    res = np.vstack([np.full(len(layout_res),layout),layout_res]).T
    true_res_data = np.vstack([true_res_data,res])

true_res_df = pd.DataFrame(true_res_data, columns=["layout", "SE"])
true_res_df.SE = pd.to_numeric(true_res_df.SE, errors='coerce')
true_res_df = true_res_df.sort_values("SE")

fig, ax = plt.subplots(figsize=(15, 8))

ax = sns.barplot(x="layout", y="SE", data=true_res_df, order=["PLAID", "RANDOM", "BORDER"])

In [ ]:
rand_results_array = true_res_df.SE[true_res_df.layout=="RANDOM"]
plaid_results_array = true_res_df.SE[true_res_df.layout=="PLAID"]
border_results_array = true_res_df.SE[true_res_df.layout=="BORDER"]

print("Variance of PLAID layouts:", statistics.variance(plaid_results_array))
print("Variance of RANDOM layouts:", statistics.variance(rand_results_array))
print("Variance of BORDER layouts:", statistics.variance(border_results_array),"\n")

print("PLAID vs RANDOM layouts:", stats.ttest_ind(plaid_results_array,rand_results_array,equal_var = False))
print("RANDOM vs BORDER layouts:", stats.ttest_ind(border_results_array,rand_results_array,equal_var = False))
print("PLAID vs BORDER layouts:", stats.ttest_ind(plaid_results_array,border_results_array,equal_var = False))

# Utilities

Examples on how to add more plate layouts

In [ ]:
# Add new PLAID layouts

layout_array = 



util.save_plaid_layout("10", layout_array, num_rows=16, num_columns=24, compounds=72, concentrations=4, replicates=1, size_empty_edge=1, neg_controls=20,directory="compounds_PLAID_layouts")

# Check if there are duplicated layouts
util.check_duplicated_layouts(layout_dir = "compounds_PLAID_layouts/")


## References

[1] 
Yann Abraham. _Dose Response Curve Fitting in Python_. Available at https://gist.github.com/yannabraham/5f210fed773785d8b638 

$\LaTeX$

In [ ]:
residuals_loaded = np.loadtxt('residuals-20210611.csv', delimiter=',', dtype='str')

In [ ]:
residuals_loaded = np.loadtxt('relative_ic50_data-20210611.csv', delimiter=',', dtype='str')